In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!pwd

/tmp/042bd4a0-d074-4dde-9ea7-06d0e8be683d


In [2]:
import configparser
import os.path
from os import path
from importlib import reload

WANDB_enable = False
creds_path_ar = ["../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

NameError: name 'ENV' is not defined

In [ ]:
cd {PATH_ROOT}

In [ ]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.utils import experiment_utils as utils
from src.utils import boilerplate

from src.model_arch import arch_setup
from src.model_arch import base_base_model
from src.model_arch import alex_model

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [8]:
train_x, train_y, val_x, val_y = boilerplate.classic_trainval(PATH_ROOT,PATH_DATA)

NameError: name 'boilerplate' is not defined

In [9]:
train_set = arch_setup.DS(train_x,train_y)
val_set= arch_setup.DS(val_x,val_y)

train_loader=DataLoader(dataset= train_set, batch_size = 16, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = 16, shuffle = True, num_workers = 2)

NameError: name 'arch_setup' is not defined

In [10]:
model= alex_model.classic_alex()
# model.apply(init_weights)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


NameError: name 'alex_model' is not defined

In [47]:
model.features[0] = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size=(11,11), stride=(4,4), padding=(2,2,))

In [62]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [63]:
arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,train_y=train_y,val_y=val_y)

started training epoch no. 1
torch.Size([1, 126, 32, 16])
torch.Size([3, 126, 32, 16])


RuntimeError: Given input size: (256x6x1). Calculated output size: (256x2x0). Output size is too small

In [ ]:
pred = [arch_setup.thresh(model(torch.from_numpy(train_x).type(torch.float32)).detach().cpu()),arch_setup.thresh(model(torch.from_numpy(val_x).type(torch.float32)).detach().cpu())]
actual = [train_y, val_y]
utils.stats(pred, actual)